In [ ]:
# Copyright 2024 Google LLC

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# YouTube Live Stream Monetization: Colab

Author: Oliva Cárcar Núñez

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.sandbox.google.com/github/YouTubeLabs/code-samples/blob/main/yt_livestream_monetization/yt_livestream_monetization.ipynb)

# Introduction

## Context

When a channel is enabled for monetization, YouTube will trigger ads to serve on
a channel content. **Mid-roll ads** are ads played during the middle of the
video, and can be automatically placed at natural breaks in your videos to
balance viewer experience and monetization potential for you.
[Help Center](https://support.google.com/youtube/answer/6175006).

**Live streams can be eligible for mid-roll ads** run during the live stream.
You can **let YouTube insert** mid-roll ads for you, **or choose how** mid-roll
ads appear. Note that when running a request for mid-roll insertion in a live
stream, it is not guaranteed that all viewers will get a mid-roll ad. If no ad
runs, viewers will continue to watch the live stream.
[Help Center](https://support.google.com/youtube/answer/7385599).

## Objective

This colab aims to provide an **automatic solution** for **YouTube Partners**
who want to **drive monetization** of their **live streams** by inserting
mid-roll ads via **API**.

## Requirements

1.  Set up a
    [**Google Cloud Platform project**](https://console.cloud.google.com/) (GCP)
    which has enabled
    [**YouTube Data API**](https://console.cloud.google.com/apis/library/youtube.googleapis.com)
    from the [API library](https://console.cloud.google.com/apis/library).
2.  Have a GCP **Service Account** with Key. The colab will required a valid
    .json file.
    [Help Center](https://cloud.google.com/iam/docs/keys-create-delete).
3.  Have a valid YouTube **Channel eligible for monetization**, and a valid
    **Content Owner** to whom the channel is associated with.
4.  Grant **access and permissions** to the GCP **Service Account** to your
    **Content Owner** and your **Channel**.
5.  A broadcast **actively live streaming** in a YPP channel.

**Note**: This Colab has been developed using Service Account credentials,
requiring Content Owner and Channel permissions; but it could be customized to
channels who do not have a YouTube Content Owner account, and to other
credential methods such as OAuth Consent Screen.

## Colab Considerations

*   Colab prioritises interactive compute. Runtimes will time out if you are
    idle or you close your browser session. In the version of Colab that is
    **free** of charge, notebooks can run for **12 hours** at most, depending on
    availability and your browser usage patterns. Check a more powerful
    [**Colab Plan**](https://colab.research.google.com/signup) (execution time
    and background running) if your needs require it.
*   If you are **new with Colab**, check out this
    [notebook](https://colab.sandbox.google.com/) to learn more about the tool.

# Authentication & Libraries

Executing the following code, **libraries are imported** and authentication
process is run. After loading the libraries, you have to **generate
credentials**, following the log instructions and uploading a **valid .json
file**.

In [ ]:
!pip3 install 'google-api-python-client==2.84.0'
!pip3 install 'google-auth==2.17.3'
!pip3 install 'google-auth-httplib2==0.1.1'
!pip3 install 'google-auth-oauthlib==1.0.0'

from datetime import datetime
import json
import time
from google.colab import files
from google.oauth2 import service_account
from googleapiclient import discovery, http

print('Successfully imported libraries!')

API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
API_SCOPES = [
    'https://www.googleapis.com/auth/youtube.force-ssl',
    'https://www.googleapis.com/auth/youtubepartner',
]

# Authenticate using user credentials stored in a .json file
service_account_file = files.upload()
service_account_json = json.loads(next(iter(service_account_file.values())))
credentials = service_account.Credentials.from_service_account_info(
    service_account_json, scopes=API_SCOPES
)
print('Success!')

# Build YT Data API service object
ytdata_service = discovery.build(
    API_SERVICE_NAME, API_VERSION, credentials=credentials
)
print('YT Data API service object created.')

# YouTube Live Stream and Ad Settings

In this section the various inputs with information from your YouTube account
and live stream are defined in order to use them as global variables during the
Colab execution.

*   **CONTENT_OWNER_ID** * - YouTube Content Owner ID.

*   **CHANNEL_ID** * - YouTube Channel ID of the channel associated with the
    broadcast into which the cuepoints are going to be inserted, e.g.:
    UCBR8-60-B28hp2BmDPdntcQ.

*   **VIDEO_ID** * : The following id parameter identifies the broadcast into
    which the cuepoints are going to be inserted. The broadcast must be actively
    streaming when inserting the cuepoint. You can define a list of video ids
    separated by commas, e.g.: Fb4AtlHKz8c,WGdiblTHG7M,9nXpWUlPM0c

*   **ADS_SLOT_DURATION_SEC**: The cuepoint's duration in seconds. The value
    must be a positive integer. You can keep 0 to not specify any Ad Slot
    Duration, the default value used to be 30 secs in that case.

*   **ADS_INTERVAL_MIN**: Request ads insertion every X minutes. By default 5.

(*) Note: Fields tagged with * are mandatory, since they are key for the
execution.

In [ ]:
# Defining the user inputs as global variables
global CONTENT_OWNER_ID
global CHANNEL_ID
global VIDEO_ID_LIST
global ADS_INTERVAL

# Content Owner ID
CONTENT_OWNER_ID = ''  # @param {type:"string"}
# Channel ID
CHANNEL_ID = ''  # @param {type:"string"}
# Video ID
VIDEO_ID = ''  # @param {type:"string"}
# The cuepoint's duration in seconds
ADS_SLOT_DURATION_SEC = 0  # @param {type:"integer"}
# Minutes interval defines for the ads
ADS_INTERVAL_MIN = 5  # @param {type:"slider", min:1, max:60, step:1}

print('YT settings saved!')

# YouTube Monetizing Live Stream Execution

Main tool execution.

In [ ]:
def inputs_validation(co, channel, video, ads_slot, ads_interval):
  """Validating the inputs defined by the user."""
  # Content Owner ID is a mandatory field
  if not co:
    raise Exception(
        'A Content Owner ID is required. Please, insert it in the Settings'
        ' section.'
    )
  # Channel ID is a mandatory field
  if not channel:
    raise Exception(
        'A Channel ID is required. Please, insert it in the Settings section.'
    )
  # Video ID of the livestream is a mandatory field
  if not video:
    raise Exception(
        'A Video ID of a live stream to identify the broadcast is needed.'
        ' Please, add it in the Settings section.'
    )
  # Ads Interval validation
  if not ads_interval:
    raise Exception(
        'Please, review Ads Interval time parameter in the Settings section.'
    )
  # Ads Slot validation
  if ads_slot is None:
    raise Exception(
        'Please, review Ads Slot Duration parameter in the Settings section,'
        ' reporting 0 (for a default set-up) or a positive integer value.'
    )
  else:
    if ads_slot < 0 or ads_slot > 3600:
      raise Exception(
          'Please, review Ads Slot Duration parameter in the Settings section,'
          ' reporting 0 (for a default set-up) or a positive integer value.'
      )
  return True


def split_video_list(video):
  """Splitting VIDEO_ID fields in a list of videos."""
  video_list = video.split(',')
  return video_list


def get_video(id):
  """Get video data."""
  # Getting video information
  videos = ytdata_service.videos().list(part='snippet', id=id).execute()

  # Return the list of video items
  return videos


def initial_live_validation(video_list):
  """Ensuring all videos are active live broadcasts, and printing some logs."""
  # Getting video information
  videos = get_video(video_list)

  # Printing video information
  for video in videos['items']:
    # Validate if the video is an active live.
    if video['snippet']['liveBroadcastContent'] == 'live':
      print(
          f"Video live {video['id']}. Title: {video['snippet']['title']}."
          f" Published: {video['snippet']['publishedAt']}."
      )
    else:
      print(
          f"Video {video['id']} is not an active live broadcast! It has been"
          ' removed from the process.'
      )
      video_list.remove(video['id'])

  # Return an updated list of videos based on video resources
  return video_list


def active_live_validation(id):
  """Checking if a video is an active live broadcast."""
  # Getting video information
  video = get_video(id)

  # Returning True if the video is an active live
  if video['items'][0]['snippet']['liveBroadcastContent'] == 'live':
    return True
  else:
    return False


def insert_ad_into_video(video, channel, co, ads_slot):
  """Inserting an ad mid-roll into a live stream."""
  # Update the live stream
  # Informing all required parameters + ad_slot
  if ads_slot > 0:
    cuepoint = (
        ytdata_service.liveBroadcasts()
        .insertCuepoint(
            id=video,
            onBehalfOfContentOwner=co,
            onBehalfOfContentOwnerChannel=channel,
            body={'durationSecs': ads_slot, 'cueType': 'cueTypeAd'},
        )
        .execute()
    )
  # Informing all required parameters and taking default YT ad_slot
  else:
    cuepoint = (
        ytdata_service.liveBroadcasts()
        .insertCuepoint(
            id=video,
            onBehalfOfContentOwner=co,
            onBehalfOfContentOwnerChannel=channel,
            body={'cueType': 'cueTypeAd'},
        )
        .execute()
    )
  current_date = datetime.now()
  print(
      f'Video {video}. {current_date.hour}:{current_date.minute} - Ad info'
      f' {cuepoint}'
  )


def validate_video_list(video_list):
  """Validate the list of videos before next iteration."""
  for video in video_list:
    if not active_live_validation(video):
      video_list.remove(video)
  return video_list


def main():
  """Main section."""
  try:
    # Validation of the global inputs before running the script
    inputs_validation(
        CONTENT_OWNER_ID,
        CHANNEL_ID,
        VIDEO_ID,
        ADS_SLOT_DURATION_SEC,
        ADS_INTERVAL_MIN,
    )
    print('Variables have been validated!')

    print('Preparing the list of videos for ads inserction.')
    # Taking a list of video ids in case there are multiple elements
    video_list = split_video_list(VIDEO_ID)
    # Updated targeted video data
    video_list = initial_live_validation(video_list)

    # Loop so that the planned list of videos keeps on running all time
    while video_list:
      # Loop the list of videos
      for video in video_list:
        insert_ad_into_video(
            video, CHANNEL_ID, CONTENT_OWNER_ID, ADS_SLOT_DURATION_SEC
        )
      time.sleep(60 * ADS_INTERVAL_MIN)
      # Checks whether the scheduled videos are active lives
      video_list = validate_video_list(video_list)

    print('Execution finished!')

  except http.HttpError as he:
    # HTTP error happened
    print(
        f'An HTTP error {he.resp.status} occurred and the API could not return'
        ' a response:'
    )
    print(he.content)
    print('Ensure correct set up of requirements and parameters.')
  except Exception as e:
    # Exception happened
    print(
        'An exception occurred when running the colab. Ensure correct set up of'
        ' requirements and parameters:'
    )
    print(e)
  except KeyboardInterrupt as k:
    # Execution stopped
    print('Execution has been stopped!')
    print(k)

In [ ]:
# Main section
if __name__ == '__main__':
  main()

# Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use
this file except in compliance with the License. You may obtain a copy of the
License at

```
https://www.apache.org/licenses/LICENSE-2.0
```

Unless required by applicable law or agreed to in writing, software distributed
under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR
CONDITIONS OF ANY KIND, either express or implied. See the License for the
specific language governing permissions and limitations under the License.